In [434]:
import warnings
warnings.filterwarnings('ignore')

In [435]:
#!/usr/bin/python3
# -*- coding: UTF-8 -*-

import os
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, SimpleRNN,Activation
from keras.layers.embeddings import Embedding
from datetime import datetime
from gensim.models import word2vec

In [436]:
input_size = int(33599 / 3)
print(input_size, 33599 / 3)

11199 11199.666666666666


In [437]:
import pandas as pd

TRAINING_PATH = './data/training/'
TESTING_PATH = './data/testing/'

categories = [dirname for dirname in os.listdir(TRAINING_PATH) if dirname[-4:] != '_cut']
print(len(categories), str(categories))

category2idx = {'Japan_Travel': 0, 'KR_ENTERTAIN': 1, 'Makeup': 2, 'Tech_Job':  3, 'WomenTalk': 4,
                  'babymother': 5, 'e-shopping': 6, 'graduate': 7, 'joke': 8, 'movie': 9}


train_list = []
for category in categories:
    category_idx = category2idx[category]
    category_path = TRAINING_PATH + category + '_cut/'
    
    for filename in os.listdir(category_path):
        filepath = category_path + filename
        
        with open(filepath, encoding='utf-8') as file:
            words = file.read().strip().split(' / ')
            train_list.append([words, category_idx])
            


10 ['Tech_Job', 'e-shopping', 'babymother', 'joke', 'Makeup', 'Japan_Travel', 'WomenTalk', 'movie', 'KR_ENTERTAIN', 'graduate']


In [438]:
test_list = []
category_path = TESTING_PATH 
    
for filename in os.listdir(category_path):
        filepath = category_path + filename
        
        with open(filepath, encoding='utf-8') as file:
            words = file.read().strip().split(' / ')
            test_list.append([words, 1])

# test_list[1]

In [439]:
# train_list[0][1]
train_dict =  dict(enumerate(train_list))
test_dict =  dict(enumerate(test_list))
# train_dict[0][1]
# len(train_dict)

In [440]:
def getArticleVec(article):
    article_vec = []
    length = 0
    for word in article[0]:
        if word in wvv.keys():
#             print(answer[word])
            article_vec.append(answer[word])
            length+=1
        else:
            temp = np.zeros(250)
            article_vec.append(temp)
#     print(len(article_vec), article_vec[0], article_vec)
    return np.mean(article_vec ,axis=0)

res1 = getArticleVec(train_list[0])
# print(res1)
print(len(res1))

250


In [441]:
label_list = np.zeros((len(train_dict), 10))
# inss = 0
for i,article in train_dict.items():
#     print(i,article[1])
#     if inss > 10:
#         break
    sec_i = article[1]
    label_list[i][sec_i] = 1

#     inss += 1


label_new_list= np.zeros((len(train_dict), 10))
label_list_i = 0
for label in label_list:
#     label_list[label_list_i] = np.reshape( label_list[label_list_i] ,(1, 10))
    
    label_new = np.reshape(label_list[label_list_i],(1, 10))
#     print(label_new.shape)
#     label_list[label_list_i] = label_new
#     print(label_list[label_list_i].shape)
    label_new_list[label_list_i] = label_new
    label_list_i += 1
    
print(label_new_list.shape)
print(label_new_list[0].shape)

(9000, 10)
(10,)


In [442]:
answer = word2vec.Word2Vec.load("word2vec1.model")
print(type(answer))

word_vectors = answer.wv
wvv = word_vectors.vocab
wvv_keys = wvv.keys()
wvv_keys_list = list(wvv_keys)
wvv_new_dict = dict(enumerate(wvv_keys_list))
# word_index = len(wvv_keys_list)

<class 'gensim.models.word2vec.Word2Vec'>


In [488]:
train_pickle_df = pd.read_pickle('train.pkl')
train_texts = train_pickle_df["text"]
train_texts = train_texts.values
# print(train_texts)
# ss = type(pickle_df["text"])
# print(ss)
# print(type(val))
# print(val[0])
# print(val[1])

In [489]:
texts = train_texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print("Found %s unique tokens" % len(word_index))
# data = pad_sequences(sequences)
# print("Shape of data tensor:" , data.shape)

# path = "word_model.model"
# model.gensim.models.Word2Vec.load(path)
embeddings_index = dict()
MAX_NB_WORDS = len(word_index)
# word_vectors = answer.wv
MAX_NB_WORDS = len(word_index)
for word,vocab_obj in answer.wv.vocab.items():
    if int(vocab_obj.index) < MAX_NB_WORDS and word == -1:
#     print(word,vocab_obj)
        embeddings_index[word] = word_vectors[word]
# del answer, word_vectors
print("Found %s word vectors." % len(embeddings_index))

OUT_SIZE = 250 
embedding_matrix = np.zeros((len(word_index) + 1, OUT_SIZE))
for word,i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        embedding_matrix[i]=np.reshape(embedding_matrix[i],(1, 250))
embedding_layer = Embedding(len(word_index) + 1, OUT_SIZE,
                            weights = [embedding_matrix],
                            input_length = 250,
                            trainable = False)

Found 232236 unique tokens
Found 0 word vectors.


In [525]:
embedding_matrix_len = len(embedding_matrix)
label_list = np.zeros((embedding_matrix_len, 10))
label_id = 0
for i,article in train_dict.items():
    article_len = len(article)
    sec_i = article[1]
    for w_i in range(0, article_len):
        label_list[label_id][sec_i] = 1
        label_id += 1
    
# label_new_list= np.zeros((embedding_matrix_len, 10))
# label_list_i = 0
# for label in label_list:
#     label_new = np.reshape(label_list[label_list_i],(1, 10))
#     label_new_list[label_list_i] = label_new
#     label_list_i += 1
    
# del label_list    
# print(label_list.shape)
# print(label_new_list[0].shape)

In [507]:
# input_dim = 400
# output_dim = 100
model = Sequential()
model.add(embedding_layer)

model.add(SimpleRNN(output_dim = 100, 
                    unroll=True))
# model.add(SimpleRNN( output_dim=50, unroll=True, ))

nb_classes = 10
model.add(Dense( nb_classes))
model.add(Activation('softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',  metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_39 (Embedding)     (None, 250, 250)          58059250  
_________________________________________________________________
simple_rnn_31 (SimpleRNN)    (None, 100)               35100     
_________________________________________________________________
dense_27 (Dense)             (None, 10)                1010      
_________________________________________________________________
activation_27 (Activation)   (None, 10)                0         
Total params: 58,095,360
Trainable params: 36,110
Non-trainable params: 58,059,250
_________________________________________________________________


In [526]:
# train_result= embedding_matrix[0:47]
# train_label = label_list[0:47]
# model.fit(x = train_result, 
#           y = train_label,
#           epochs = 5, 
#           verbose = 0)
print(len(embedding_matrix))
print(len(label_list))

232237
232237


In [ ]:
import time

start_time = time.time()
start = 0
embedding_matrix
for embed_item_index in range(0, 487):
#     train_result= np.asarray([embedding_matrix])
#     train_label =  np.asarray([label_list[label_id]])
    
    end = start + 100
    train_result= embedding_matrix[start:end]
    train_label = label_list[start:end]
    model.fit(x = train_result, y = train_label,
              epochs = 5, verbose = 0)
    
    scores = model.evaluate(x = train_result, y = train_label, batch_size=10, verbose = 0)
    if  embed_item_index % 10 == 0 :  
        end_time = time.time()
        excution_time = end_time- start_time
        excution_time_min = round(excution_time / 60, 2)
        print(embed_item_index, " , loss : ", round(scores[0], 6), "   acc : " , scores[1],
              "  minutes: " , excution_time_min)
        
    if embed_item_index < 400 :  
        pass
        
    else:
        break
    start += 100


0  , loss :  3.32766    acc :  0.0   minutes:  0.02
10  , loss :  0.006115    acc :  1.0   minutes:  0.25
20  , loss :  0.044049    acc :  1.0   minutes:  0.47
30  , loss :  0.005844    acc :  1.0   minutes:  0.69
40  , loss :  0.007404    acc :  1.0   minutes:  0.92
50  , loss :  0.003341    acc :  1.0   minutes:  1.14
60  , loss :  0.000616    acc :  1.0   minutes:  1.36
70  , loss :  0.000555    acc :  1.0   minutes:  1.59
80  , loss :  0.00018    acc :  1.0   minutes:  1.81
90  , loss :  3.999848    acc :  0.0   minutes:  2.03
100  , loss :  0.008296    acc :  1.0   minutes:  2.26
110  , loss :  0.000381    acc :  1.0   minutes:  2.48


In [506]:
test_feature=  embedding_matrix[5500:5501]
predict_res = model.predict(test_feature, batch_size= 10, verbose=0)
#     print(len(predict_res), predict_res[0])
final_res = []
now_item = predict_res[0]
max_val = max(now_item)
final_res.append(np.argmax(now_item))
print(final_res)

[7]


In [ ]:
answer = word2vec.Word2Vec.load("word2vec1.model")
word_vectors = answer.wv
wvv = word_vectors.vocab
wvv_keys = wvv.keys()
wvv_keys_list = list(wvv_keys)
wvv_new_dict1 = dict(enumerate(wvv_keys_list))
print(type(wvv_new_dict1))
test_embedding_matrix = np.zeros((len(test_dict) + 1, 400))
# inss = 0
for i,word in test_dict.items():
#     print(i,train_dict[i][1], word, "\n")
#     if inss > 10:
#         break
    test_embedding_vector = wvv_new_dict1[word]
    test_embedding_matrix[i] = test_embedding_vector
# del answer
# embeddings_index = list()
# # word_vectors = answer.wv
# MAX_NB_WORDS = len(word_index)
# for word,vocab_obj in word_vectors.vocab.items():
#     if int(vocab_obj.index) < MAX_NB_WORDS:
#         embeddings_index[word] = word_vectors[word]
# del answer, word_vectors
# print("Found %s word vectors." % len(embeddings_index))

In [ ]:
final_res = []
print(len(test_embedding_matrix))
for predicc in test_embedding_matrix:
    test_feature= np.asarray(predicc)
    predict_res = model.predict(test_feature, batch_size= 10, verbose=0)
#     print(len(predict_res), predict_res[0])
    
    now_item = predict_res[0]
    max_val = max(now_item)
    final_res.append(np.argmax(now_item))
print(final_res)

In [ ]:
test_array = np.asarray(test_list)
print(test_array.shape)

In [ ]:
# result_txt = "result" + str(datetime.now()).split()[1] + ".txt"
print(len(final_res))
result_txt = "result001" + ".txt"
ids = 0
with open(result_txt, 'a') as out:
    out.write("id,category" + '\n')
    for value in final_res:
        out.write(str(ids) + "," + str(value) + '\n')
        ids += 1